# Semantic Text Similarity
Este modelo utiliza gensim para convertir pares de vectores + puntuaciones en vectores (word embeddings).
Dado un dataset, infiere la puntuación de similitud entre ambas frases.

In [1]:
# Requisitos
from gensim.models import TfidfModel
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np

In [2]:
# Tipado
from typing import Tuple, List

In [3]:
# Cargar stopwords en Catalan
# STOPWORDS_CA = {"a", "abans", "ací", "ah", "així", "això", "al", "aleshores", "algun", "alguna", "algunes", "alguns", "alhora", "allà", "allí", "allò", "als", "altra", "altre", "altres", "amb", "ambdues", "ambdós", "anar", "ans", "apa", "aquell", "aquella", "aquelles", "aquells", "aquest", "aquesta", "aquestes", "aquests", "aquí", "baix", "bastant", "bé", "cada", "cadascuna", "cadascunes", "cadascuns", "cadascú", "com", "consegueixo", "conseguim", "conseguir", "consigueix", "consigueixen", "consigueixes", "contra", "d'un", "d'una", "d'unes", "d'uns", "dalt", "de", "del", "dels", "des", "des de", "després", "dins", "dintre", "donat", "doncs", "durant", "e", "eh", "el", "elles", "ells", "els", "em", "en", "encara", "ens", "entre", "era", "erem", "eren", "eres", "es", "esta", "estan", "estat", "estava", "estaven", "estem", "esteu", "estic", "està", "estàvem", "estàveu", "et", "etc", "ets", "fa", "faig", "fan", "fas", "fem", "fer", "feu", "fi", "fins", "fora", "gairebé", "ha", "han", "has", "haver", "havia", "he", "hem", "heu", "hi", "ho", "i", "igual", "iguals", "inclòs", "ja", "jo", "l'hi", "la", "les", "li", "li'n", "llarg", "llavors", "m'he", "ma", "mal", "malgrat", "mateix", "mateixa", "mateixes", "mateixos", "me", "mentre", "meu", "meus", "meva", "meves", "mode", "molt", "molta", "moltes", "molts", "mon", "mons", "més", "n'he", "n'hi", "ne", "ni", "no", "nogensmenys", "només", "nosaltres", "nostra", "nostre", "nostres", "o", "oh", "oi", "on", "pas", "pel", "pels", "per", "per que", "perquè", "però", "poc", "poca", "pocs", "podem", "poden", "poder", "podeu", "poques", "potser", "primer", "propi", "puc", "qual", "quals", "quan", "quant", "que", "quelcom", "qui", "quin", "quina", "quines", "quins", "què", "s'ha", "s'han", "sa", "sabem", "saben", "saber", "sabeu", "sap", "saps", "semblant", "semblants", "sense", "ser", "ses", "seu", "seus", "seva", "seves", "si", "sobre", "sobretot", "soc", "solament", "sols", "som", "son", "sons", "sota", "sou", "sóc", "són", "t'ha", "t'han", "t'he", "ta", "tal", "també", "tampoc", "tan", "tant", "tanta", "tantes", "te", "tene", "tenim", "tenir", "teniu", "teu", "teus", "teva", "teves", "tinc", "ton", "tons", "tot", "tota", "totes", "tots", "un", "una", "unes", "uns", "us", "va", "vaig", "vam", "van", "vas", "veu", "vosaltres", "vostra", "vostre", "vostres", "érem", "éreu", "és", "éssent", "últim", "ús"}
STOPWORDS_CA = {"a", "al", "el", "la", "els", "les", "de", "un", "una", "algun", "alguna", }

In [5]:
# Definir función de pre-procesado
def preprocess(sentence: str) -> List[str]:
    preprocessed = simple_preprocess(sentence)
    preprocessed = [token for token in preprocessed if token not in STOPWORDS_CA]
    return preprocessed

In [6]:
# Modelos pre-entrenados
# WV_MODEL_PATH = "/Users/salva/Downloads/cc.ca.300.bin.gz"
WV_MODEL_PATH = '/Users/salva/Downloads/cc.ca.300.vec.gz'
import gensim
wv_model =  gensim.models.KeyedVectors.load_word2vec_format(WV_MODEL_PATH, binary=False)
wv_model

In [8]:
# Ejemplo de 10 pares de oraciones con puntuación de similitud asociada
input_pairs = [
    ('M\'agrada el futbol', 'Disfruto veient partits de futbol', 4),
    ('El cel està despejat', 'Fa un dia bonic', 4.5),
    ('M\'encanta viatjar', 'Explorar nous llocs és una passió', 3.5),
    ('Prefereixo l\'estiu', 'No m\'agrada el fred de l\'hivern', 2.5),
    ('Tinc gana', 'Què hi ha per sopar?', 2),
    ('La música em relaxa', 'Escoltar música és una teràpia', 3),
    ('El llibre és emocionant', 'No puc deixar de llegir-lo', 4),
    ('M\'agrada la pizza', 'És el meu menjar preferit', 4.5),
    ('Estic cansat', 'Necessito fer una migdiada', 1.5),
    ('Avui fa molta calor', 'És un dia sofocant', 3.5)
    ]

In [9]:
from datasets import load_dataset
# Text Similarity (STS) dataset (principal per la Pràctica 4)
train = load_dataset("projecte-aina/sts-ca", split="train")
test = load_dataset("projecte-aina/sts-ca", split="test")
val = load_dataset("projecte-aina/sts-ca", split="validation")
all_data = load_dataset("projecte-aina/sts-ca", split="all")
all_data

Dataset({
    features: ['id', 'sentence_1', 'sentence_2', 'label'],
    num_rows: 3073
})

In [10]:
# Preprocesamiento de las oraciones y creación del diccionario
sentences_1_preproc = [simple_preprocess(d["sentence_1"]) for d in all_data]
sentences_2_preproc = [simple_preprocess(d["sentence_2"]) for d in all_data]
scores = [d["label"] for d in all_data]
sentence_pairs = list(zip(sentences_1_preproc, sentences_2_preproc, scores))
# Versión aplanada para poder entrenar el modelo
sentences_pairs_flattened = sentences_1_preproc + sentences_2_preproc
diccionario = Dictionary(sentences_pairs_flattened)
diccionario

In [11]:
print(sentence_pairs[0])

(['atorga', 'per', 'primer', 'cop', 'les', 'mencions', 'encarna', 'sanahuja', 'la', 'inclusió', 'de', 'la', 'perspectiva', 'de', 'gènere', 'en', 'docència', 'universitària'], ['creen', 'la', 'menció', 'encarna', 'sanahuja', 'la', 'inclusió', 'de', 'la', 'perspectiva', 'de', 'gènere', 'en', 'docència', 'universitària'], 3.5)


In [12]:
# Cálculo de los pesos TF-IDF para las oraciones pre-procesadas
corpus = [diccionario.doc2bow(sent) for sent in sentences_pairs_flattened]
modelo_tfidf = TfidfModel(corpus)

In [15]:
def map_tf_idf(sentence_preproc: List[str], dictionary: Dictionary, tf_idf_model: TfidfModel) -> Tuple[List[np.ndarray], List[float]]:
    bow = dictionary.doc2bow(sentence_preproc)
    tf_idf = tf_idf_model[bow]
    vectors, weights = [], []
    for word_index, weight in tf_idf:
        word = dictionary.get(word_index)
        if word in wv_model:
            vectors.append(wv_model[word])
            weights.append(weight)
    return vectors, weights

def map_pairs(
        sentence_pairs: List[Tuple[str, str, float]],
        dictionary: Dictionary = None,
        tf_idf_model: TfidfModel = None,
) -> List[Tuple[Tuple[np.ndarray, np.ndarray], float]]:
    """
    Mapea los tripletes de oraciones a listas de (x, y), (pares de vectores, score)
    :param sentence_pairs:
    :param dictionary:
    :param tf_idf_model:
    :return:
    """
    # Mapeo de los pares de oraciones a pares de vectores
    pares_vectores = []
    for i, (sentence_1, sentence_2, similitud) in enumerate(sentence_pairs):
        sentence_1_preproc = preprocess(sentence_1) if isinstance(sentence_1, str) else sentence_1
        sentence_2_preproc = preprocess(sentence_2) if isinstance(sentence_2, str) else sentence_2
        # Si usamos TF-IDF
        if tf_idf_model is not None:
            # Cálculo del promedio ponderado por TF-IDF de los word embeddings
            vectors1, weights1 = map_tf_idf(sentence_1_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model, )
            vectors2, weights2 = map_tf_idf(sentence_2_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model, )
            vector1 = np.average(vectors1, weights=weights1, axis=0, )
            vector2 = np.average(vectors2, weights=weights2, axis=0, )
        else:
            # Cálculo del promedio de los word embeddings
            vectors1 = [wv_model[word] for word in sentence_1_preproc if word in wv_model]
            vectors2 = [wv_model[word] for word in sentence_2_preproc if word in wv_model]
            vector1 = np.mean(vectors1, axis=0)
            vector2 = np.mean(vectors2, axis=0)
        # Añadir a la lista
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores

In [16]:
# Imprimir los pares de vectores y la puntuación de similitud asociada
mapped = map_pairs(sentence_pairs, tf_idf_model=modelo_tfidf, dictionary=diccionario, )
mapped[0]

((array([-7.22131877e-03, -4.88683421e-03,  2.71017708e-02,  2.32332627e-02,
         -9.60097482e-03, -3.16095435e-03,  2.90225599e-02, -1.75413820e-02,
          2.93095319e-02, -1.60574403e-02, -6.04936805e-03,  1.49908545e-02,
          1.00934507e-02,  1.84449753e-02,  2.16156266e-02,  2.13238810e-02,
          8.69774586e-03,  6.13958934e-02,  2.18719114e-02,  1.01426407e-02,
          1.16837708e-02,  3.24588305e-03, -1.32856906e-02,  5.77104877e-02,
          1.54627187e-02,  2.13443526e-02, -3.82471218e-02,  6.23983637e-03,
          7.31161386e-04,  8.99225741e-03, -4.87626204e-03,  1.08773269e-02,
          1.30313145e-02, -3.86450925e-03,  7.23370200e-03, -1.75266524e-02,
         -9.09778208e-03,  4.43412138e-02, -4.31998409e-04,  6.25044253e-04,
         -1.13920750e-02, -1.82465011e-02, -8.11444328e-03, -8.18518457e-03,
         -3.54176235e-03, -1.60820262e-01,  7.74505797e-03,  9.80261699e-03,
          8.53058034e-03, -1.23878019e-02,  1.24134202e-02, -2.39070017e-03,

In [17]:
# Definir el Modelo
import tensorflow as tf

def build_and_compile_model(hidden_size: int = 128, embedding_size: int = 300, learning_rate: float = 0.001) -> tf.keras.Model:
    # Capa de entrada para los pares de vectores
    input_1 = tf.keras.Input(shape=(embedding_size,))
    input_2 = tf.keras.Input(shape=(embedding_size,))

    # Capa oculta
    first_projection = tf.keras.layers.Dense(
        embedding_size,
        # activation='tanh',
        kernel_initializer=tf.keras.initializers.Identity(),
        bias_initializer=tf.keras.initializers.Zeros(),
    )
    projected_1 = first_projection(input_1)
    projected_2 = first_projection(input_2)

    # Compute the cosine distance
    projected_1 = tf.linalg.l2_normalize(projected_1, axis=1, )
    projected_2 = tf.linalg.l2_normalize(projected_2, axis=1, )
    output = 2.5 * (1.0 + tf.reduce_sum(projected_1 * projected_2, axis=1, ))

    # Definir el modelo con las capas de entrada y salida
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

    # Compilar el modelo
    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(learning_rate))

    return model

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [18]:
# Definir constantes de entrenamiento
batch_size: int = 64
num_epochs: int = 64
train_val_split: float = 0.8

In [50]:
# Obtener x_train e y_train
train_slice: int = int(len(mapped) * train_val_split)

def pair_list_to_x_y(pair_list: List[Tuple[Tuple[np.ndarray, np.ndarray], int]]) -> Tuple[Tuple[np.ndarray, np.ndarray], np.ndarray]:
    """
    Otiene las matrices X_1 (N x d) , X_2 (N x d), e Y (n) a partir de listas de parejas de vectores de oraciones - Listas de (d, d, 1)
    :param pair_list:
    :return:
    """
    _x, _y = zip(*pair_list)
    _x_1, _x_2 = zip(*_x)
    return (np.array(_x_1), np.array(_x_2)), np.array(_y, dtype=np.float32, ) / 5.0

# Obtener las listas de train y test
x_train, y_train = pair_list_to_x_y(mapped[:train_slice])
x_val, y_val = pair_list_to_x_y(mapped[train_slice:])

In [51]:
# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [102]:

def build_and_compile_model(embedding_size: int = 300, learning_rate: float = 0.001) -> tf.keras.Model:
    # Input layer
    input_1 = tf.keras.Input(shape=(embedding_size,), name="input_vector_1")
    input_2 = tf.keras.Input(shape=(embedding_size,), name="input_vector_2")

    # hidden layer
    first_projection_layer = tf.keras.layers.Dense(
        embedding_size,
        activation='tanh',
        kernel_initializer=tf.keras.initializers.Identity(),
        bias_initializer=tf.keras.initializers.Zeros(),
        name="projection_layer"
    )
    dropout = tf.keras.layers.Dropout(0.3, name="projection_dropout")
    projected_1_dense = dropout(first_projection_layer(input_1))
    projected_2_dense = dropout(first_projection_layer(input_2))

    # Normalize the projected vectors using Lambda layers
    normalized_1 = tf.keras.layers.Lambda(
        lambda x: tf.linalg.l2_normalize(x, axis=1), name="normalize_1"
    )(projected_1_dense)
    normalized_2 = tf.keras.layers.Lambda(
        lambda x: tf.linalg.l2_normalize(x, axis=1), name="normalize_2"
    )(projected_2_dense)

    # Compute the custom similarity score using a Lambda layer
    similarity_sum = tf.keras.layers.Lambda(
        lambda x: tf.reduce_sum(x[0] * x[1], axis=1, keepdims=True), name="similarity_sum"
    )([normalized_1, normalized_2])

    output = tf.keras.layers.Lambda(
        lambda x: 0.5 * (1.0 + x), name="output_scaling"
    )(similarity_sum)

    # Definir el modelo con las capas de entrada y salida
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output, name="similarity_model")

    # Compilar el modelo
    model.compile(
        loss='mean_squared_error',
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    )

    return model


In [103]:
# Construir y compilar el modelo
model = build_and_compile_model()
# tf.keras.utils.plot_model(model, show_shapes=True, show_layer_activations=True, )
print(model.summary())
# Entrenar el modelo
model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

Model: "similarity_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_vector_1      │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_vector_2      │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ projection_layer    │ (None, 300)       │     90,300 │ input_vector_1[0… │
│ (Dense)             │                   │            │ input_vector_2[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ projection_dropout  │ (None, 300)       │          0 │ projection_layer… │
│ (Dropout)           │                   │            │ projection_layer… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalize_1         │ (None, 300)       │          0 │ projection_dropo… │
│ (Lambda)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalize_2         │ (None, 300)       │          0 │ projection_dropo… │
│ (Lambda)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ similarity_sum      │ (None, 1)         │          0 │ normalize_1[0][0… │
│ (Lambda)            │                   │            │ normalize_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_scaling      │ (None, 1)         │          0 │ similarity_sum[0… │
│ (Lambda)            │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 90,300 (352.73 KB)

 Trainable params: 90,300 (352.73 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.1008 - val_loss: 0.1401
Epoch 2/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0815 - val_loss: 0.1305
Epoch 3/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0753 - val_loss: 0.1252
Epoch 4/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0710 - val_loss: 0.1209
Epoch 5/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0672 - val_loss: 0.1186
Epoch 6/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0644 - val_loss: 0.1167
Epoch 7/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0625 - val_loss: 0.1151
Epoch 8/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0595 - val_loss: 0.1138
Epoch 9/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0577 - val_loss: 0.1125
Epoch 10/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0557 - val_loss: 0.1120
Epoch 11/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0542 - val_loss: 0.1113
Epoch 12/64
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0523 - v

In [104]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
from scipy.stats import pearsonr
from scipy import spatial
val = mapped[train_slice:]
y_pred_baseline = []
for (v1, v2), dist in val:
    d = 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlation, _ = pearsonr(np.array(y_pred_baseline), y_val.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlation}")

Correlación de Pearson: 0.43930761675943547


In [105]:
from scipy.stats import pearsonr
# Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
y_pred: tf.RaggedTensor = model.predict(x_val)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlation, _ = pearsonr(y_pred.flatten(), y_val.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlation}")


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Correlación de Pearson: 0.5313218622093188


In [106]:
from scipy.stats import pearsonr
# Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
y_pred: tf.RaggedTensor = model.predict(x_train)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlation, _ = pearsonr(y_pred.flatten(), y_train.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlation}")

77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 619us/step
Correlación de Pearson: 0.7707306954929074
